In [2]:
#use Jupyter cell magic to load all of the relevant modules
%run ../wrapper/imports.py
%config InlineBackend.figure_format='retina'

### Load Planet
Load initial guess from which the relaxation code will ramp to your desired parameters. `expedite=False` takes the time to compute all of the post-facto variables. This is not necessary to run the ramping.

In [3]:
sim = wind_sim()
sim.load_planet('saves/start_HotJupiter_H-He.csv',expedite=False)
sim.system.print_system()

Nspecies has changed from 2 to 1. Remaking C code...                                                                                      Atmosphere Composition
  Species:   HI
  Mass frac: 1.00e+00
Loaded Planet:
  System parameters (cgs)               # Normalized units
    Mp:        1.330000e+30 g           #     0.70 MJ
    Rp:        1.000000e+10 cm          #     1.40 RJ
    Mstar:     1.988416e+33 g           #     1.00 Msun
    semimajor: 7.480000e+11 cm          #     0.05 au
    Ftot:      1.095713e+03 erg/cm^2/s  # 2.35e+02 FuvEarth
    Lstar:     6.807420e+33 erg/s       # 1.78e+00 Lsun



### Define a system to ramp to

In [ ]:
HD